<a href="https://colab.research.google.com/github/alinaalborova/sentence-transformers/blob/master/sentence_embeddings_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U sentence-transformers

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:22<00:00, 17.7MB/s]


In [0]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']
sentence_embeddings = model.encode(sentences)

In [4]:
for sentence, embedding in zip(sentences, sentence_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", len(embedding), embedding[:10])
    print("")

Sentence: This framework generates embeddings for each input sentence
Embedding: 768 [-0.10409481  0.5274769   1.1797731   0.1943658   0.15911368  0.5455507
  0.28180978  0.12159055  0.2919681  -0.33685815]

Sentence: Sentences are passed as a list of string.
Embedding: 768 [-0.13118427 -0.1739028   1.1052184   0.24854213  0.04100079  0.4031433
 -0.38167295  0.1746835  -0.5365212   0.3570098 ]

Sentence: The quick brown fox jumps over the lazy dog.
Embedding: 768 [-0.74899274  0.71891785 -1.0394572   0.06408814  0.05070471 -0.7290621
 -0.37650588 -0.35532534 -0.3260914  -0.8520532 ]



In [0]:
sentence_embeddings = model.encode([
                                    'Мама мыла раму'
])

In [6]:
rus_sentences = ['мама мыла раму', 'рама мыла маму']
rus_sentences_embeddings = model.encode(rus_sentences)
for sentence, embedding in zip(rus_sentences, rus_sentences_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", len(embedding), embedding[:10])
    print("")

Sentence: мама мыла раму
Embedding: 768 [ 0.02418204  0.18126722  1.2029917   0.04019699  0.8840121   0.5068428
  0.28139967  0.4999066  -0.38268703  0.39505318]

Sentence: рама мыла маму
Embedding: 768 [ 0.01930108  0.1501914   1.1643057   0.05611549  0.83182675  0.5418704
  0.2924847   0.5805235  -0.36399835  0.33721417]



In [7]:
tatar_sentences = ['Вахит Имамовның бу китабын экстремистик китап дип бәяләргә тырышып, тыю өчен суд-мәхкәмә эшләре бара.', 
                   'Суд киләсе елда эшен дәвам итәчәк.', 
                   'Ә әлегә документаль әсәр экспертиза уза.', 
                   'Әлеге китапны ни сәбәпледер, мин дә үз вакытында укымый калганмын']
tat_sentences_embeddings = model.encode(tatar_sentences)
for sentence, embedding in zip(tatar_sentences, tat_sentences_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", len(embedding), embedding[:10])
    print("")             

Sentence: Вахит Имамовның бу китабын экстремистик китап дип бәяләргә тырышып, тыю өчен суд-мәхкәмә эшләре бара.
Embedding: 768 [-0.19085978  0.53773636  1.1533415   0.36398053  0.9467713  -0.22634296
  0.6719887   0.23747857 -0.02229607  0.29271248]

Sentence: Суд киләсе елда эшен дәвам итәчәк.
Embedding: 768 [-0.1763601   0.20949376  1.18878    -0.07288632  0.39875254 -0.01915785
  0.76020795  0.43875942 -0.05098988  0.0717279 ]

Sentence: Ә әлегә документаль әсәр экспертиза уза.
Embedding: 768 [-0.338233    0.26497146  1.0133278   0.49888176  1.0454831  -0.17264396
  0.46522376  0.51791     0.02218278  0.23018543]

Sentence: Әлеге китапны ни сәбәпледер, мин дә үз вакытында укымый калганмын
Embedding: 768 [-0.35848117  0.48103228  0.9773034   0.37314257  0.78438175 -0.09250664
  0.64860755  0.5562376  -0.32417208  0.4064436 ]



# FAISS Installation

Sadly, it can be painfull :(

In according to answer from SO: https://stackoverflow.com/questions/47967252/installing-faiss-on-google-colaboratory

In [0]:
#!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
#!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!wget https://anaconda.org/pytorch/faiss-gpu/1.2.1/download/linux-64/faiss-gpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-gpu-1.2.1-py36_cuda9.0.176_1.tar.bz2

In [9]:
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

In [0]:
import faiss

Now time for downloading datasets

In [0]:
import math
import logging
from datetime import datetime
from pathlib import Path
import os
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import joblib
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import *
from sentence_transformers.util import batch_to_device
from sentence_transformers.readers.InputExample import InputExample

In [0]:
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

# Tatoeba manipulation

In [13]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/moses/ru-tt.txt.zip
!unzip ru-tt.txt.zip -d ./tatoeba/

--2020-01-23 17:04:01--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/moses/ru-tt.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 226705 (221K) [application/zip]
Saving to: ‘ru-tt.txt.zip’

ru-tt.txt.zip       100%[===================>] 221.39K   410KB/s    in 0.5s    

2020-01-23 17:04:03 (410 KB/s) - ‘ru-tt.txt.zip’ saved [226705/226705]

Archive:  ru-tt.txt.zip
  inflating: ./tatoeba/README        
  inflating: ./tatoeba/LICENSE       
  inflating: ./tatoeba/Tatoeba.ru-tt.ru  
  inflating: ./tatoeba/Tatoeba.ru-tt.tt  
  inflating: ./tatoeba/Tatoeba.ru-tt.xml  


In [14]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/moses/en-tt.txt.zip
!unzip en-tt.txt.zip -d ./tatoeba/


--2020-01-23 17:04:06--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/moses/en-tt.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62721 (61K) [application/zip]
Saving to: ‘en-tt.txt.zip’

en-tt.txt.zip       100%[===================>]  61.25K  --.-KB/s    in 0.1s    

2020-01-23 17:04:07 (459 KB/s) - ‘en-tt.txt.zip’ saved [62721/62721]

Archive:  en-tt.txt.zip
replace ./tatoeba/README? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./tatoeba/LICENSE? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
  inflating: ./tatoeba/Tatoeba.en-tt.en  
  inflating: ./tatoeba/Tatoeba.en-tt.tt  
  inflating: ./tatoeba/Tatoeba.en-tt.xml  


In [0]:
class TatoebaReader:
    """Reads in a plain text file, in which every line contains one 
    sentence."""
    def __init__(self, file_path: Path):
        self.file_path = file_path

    def get_examples(self):
        examples = []        
        with open(self.file_path) as fin:
            for i, line in enumerate(fin.readlines()):
                examples.append(InputExample(guid=i, texts=[line], label=0))                
        return examples

TATOEBA_PATH = Path("./tatoeba/")

In [0]:

# Use BERT for mapping tokens to embeddings
# handle the downloading and caching for you:
word_embedding_model = models.BERT('bert-base-multilingual-cased')

In [0]:
def children(m):
    return m if isinstance(m, (list, tuple)) else list(m.children())


def set_trainable_attr(m, b):
    m.trainable = b
    for p in m.parameters():
        p.requires_grad = b


def apply_leaf(m, f):
    c = children(m)
    if isinstance(m, nn.Module):
        f(m)
    if len(c) > 0:
        for l in c:
            apply_leaf(l, f)


def set_trainable(l, b):
    apply_leaf(l, lambda m: set_trainable_attr(m, b))

In [18]:
set_trainable(word_embedding_model.bert.embeddings.word_embeddings, False)
print(word_embedding_model.bert.embeddings.word_embeddings.weight.requires_grad)
print(word_embedding_model.bert.embeddings.position_embeddings.weight.requires_grad)

False
True


In [0]:

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [0]:
batch_size = 16
TATOEBA_PATH = '/content/tatoeba'

In [0]:
lang_1 = 'ru'
lang_2 = 'tt'

In [0]:

def evaluate_language_pair(model, pair_name="en-tt", batch_size=32):
    lang_1, lang_2 = pair_name.split("-")
    reader_1 = TatoebaReader(os.path.join(TATOEBA_PATH, f"Tatoeba.{pair_name}.{lang_1}"))
    ds_1 = SentencesDataset(reader_1.get_examples(), model=model)
    loader_1 = DataLoader(
        ds_1, shuffle=False, batch_size=batch_size, 
        collate_fn=model.smart_batching_collate)
    reader_2 = TatoebaReader(os.path.join(TATOEBA_PATH, f"Tatoeba.{pair_name}.{lang_2}"))    
    ds_2 = SentencesDataset(reader_2.get_examples(), model=model)
    loader_2 = DataLoader(
        ds_2, shuffle=False, batch_size=batch_size, 
        collate_fn=model.smart_batching_collate)
    
    model.eval()
    emb_1, emb_2 = [], []
    with torch.no_grad():
        for batch in loader_1:
            emb_1.append(model(
                batch_to_device(batch, "cuda")[0][0]
            )['sentence_embedding'])
        for batch in loader_2:
            emb_2.append(model(
                batch_to_device(batch, "cuda")[0][0]
            )['sentence_embedding'])
    emb_1 = torch.cat(emb_1).cpu().numpy()
    emb_2 = torch.cat(emb_2).cpu().numpy()
    
    idx_1 = faiss.IndexFlatL2(emb_1.shape[1])
    faiss.normalize_L2(emb_1)
    idx_1.add(emb_1)
    idx_2 = faiss.IndexFlatL2(emb_2.shape[1])
    faiss.normalize_L2(emb_2)
    idx_2.add(emb_2)
    
    results = []
    _, match = idx_2.search(x=emb_1, k=1)
    results.append((
        lang_1, lang_2,
        np.sum(match[:, 0] == np.arange(len(emb_1))),
        len(emb_1)
    ))
    _, match = idx_1.search(x=emb_2, k=1)
    results.append((
        lang_2, lang_1,
        np.sum(match[:, 0] == np.arange(len(emb_2))),
        len(emb_2)
    ))
    return results

In [0]:
PAIRS = ["en-tt", "ru-tt"]

In [24]:
results = []
for pair in PAIRS:
    results += evaluate_language_pair(model, pair_name=pair, batch_size=50)
df_baseline_mean = pd.DataFrame(results, columns=["from", "to", "correct", "total"])
df_baseline_mean

,from,to,correct,total
0,en,tt,134,1414
1,tt,en,123,1414
2,ru,tt,561,3105
3,tt,ru,521,3105


# Fine Tuning 

Firstly, just download datasets for fine-tuning.

In [0]:
import os
folder_path = './datasets/'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)


In [26]:
import urllib.request
import zipfile
print('Beginning download of datasets')

datasets = ['AllNLI.zip', 'stsbenchmark.zip', 'wikipedia-sections-triplets.zip']
server = "https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/datasets/"

for dataset in datasets:
    print("Download", dataset)
    url = server+dataset
    dataset_path = os.path.join(folder_path, dataset)
    urllib.request.urlretrieve(url, dataset_path)

    print("Extract", dataset)
    with zipfile.ZipFile(dataset_path, "r") as zip_ref:
        zip_ref.extractall(folder_path)
    os.remove(dataset_path)


print("All datasets downloaded and extracted")

Beginning download of datasets
Download AllNLI.zip
Extract AllNLI.zip
Download stsbenchmark.zip
Extract stsbenchmark.zip
Download wikipedia-sections-triplets.zip
Extract wikipedia-sections-triplets.zip
All datasets downloaded and extracted


In [0]:

# Read the dataset
batch_size = 16
nli_reader = NLIDataReader('./datasets/AllNLI')
sts_reader = STSDataReader('./datasets/stsbenchmark')
train_num_labels = nli_reader.get_num_labels()
model_save_path = 'output/training_nli_bert-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [0]:
# Convert the dataset to a DataLoader ready for training
logging.info("Read AllNLI train dataset")
train_data = SentencesDataset(nli_reader.get_examples('train.gz', max_examples=100000), model=model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
train_loss = losses.SoftmaxLoss(
    model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=train_num_labels)


In [29]:
joblib.dump(train_data, "allnli_train_dataset.jl")

['allnli_train_dataset.jl']

In [0]:

logging.info("Read STSbenchmark dev dataset")
dev_data = SentencesDataset(examples=sts_reader.get_examples('sts-dev.csv'), model=model)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=batch_size)
evaluator = EmbeddingSimilarityEvaluator(dev_dataloader)

In [31]:
joblib.dump(dev_data, "sts_dev_dataset.jl")

['sts_dev_dataset.jl']

In [32]:
# Configure the training
num_epochs = 1

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path
          )

Epoch: 100%|██████████| 1/1 [23:12<00:00, 1392.31s/it]


# After fine-tuning

In [0]:
#model = SentenceTransformer('/content/output/training_nli_bert-2020-01-13_14-53-24')
model = SentenceTransformer('/content/output/training_nli_bert-2020-01-23_17-06-00')

In [0]:
results = []
for pair in PAIRS:
    results += evaluate_language_pair(model, pair_name=pair, batch_size=50)
df_finetuned = pd.DataFrame(results, columns=["from", "to", "correct", "total"])

In [36]:
df_finetuned

,from,to,correct,total
0,en,tt,133,1414
1,tt,en,153,1414
2,ru,tt,530,3105
3,tt,ru,549,3105


In [37]:
df_baseline_mean

,from,to,correct,total
0,en,tt,134,1414
1,tt,en,123,1414
2,ru,tt,561,3105
3,tt,ru,521,3105
